In [3]:
import pymysql
import pandas as pd

In [24]:
%%time
db = pymysql.connect(
        host = 'solring-pcs-datawarehouse-2020-11-20.cluster-cakfueupwjvh.eu-central-1.rds.amazonaws.com',
        user="root",
        password="68sP39Qn", 
        port=3306
      
    )
# Run  query in the database to join tables 
try:
    df = pd.read_sql('select  * \
         from pcs_data_warehouse.pcs_fact_flight_segments ffs \
         inner join pcs_data_warehouse.pcs_dim_bookings pdb on ffs.booking_id = pdb.id \
         inner join pcs_data_warehouse.pcs_dim_ancillaries_groups pag on ffs.ancillaries_group_id = pag.id \
         inner join pcs_data_warehouse.pcs_dim_flights pf on ffs.flight_id = pf.id \
         inner join pcs_data_warehouse.pcs_ancillaries pa on ffs.ancillaries_group_id = pa.ancillaries_group_id \
         inner join pcs_data_warehouse.pcs_dim_profiles pp on ffs.profile_id = pp.id limit 1000',
         db)
finally:
    db.close()
print(df.shape)
 
print("YAYY!!")

(1000, 90)
YAYY!!
CPU times: user 361 ms, sys: 12 ms, total: 373 ms
Wall time: 451 ms


In [25]:
df.to_csv('DataWearHouse.csv')

In [26]:
df.columns

Index(['id', 'pnr', 'first_name', 'last_name', 'company', 'flight_number',
       'departure_date', 'uci', 'DID', 'segment_number', 'cabin_class_code',
       'cabin_class', 'fare_basis', 'tkne', 'rbd', 'status_code', 'status',
       'meal_service', 'baggage_service', 'seat_service', 'company_id',
       'agency_id', 'profile_id', 'booking_id', 'passenger_id', 'route_id',
       'flight_id', 'airplane_id', 'contacts_group_id', 'ancillaries_group_id',
       'origin_airport_id', 'destination_airport_id', 'fare_family', 'issued',
       'ps_segment_number', 'id', 'pnr', 'creation_date', 'creation_time',
       'total_flight_segments', 'total_adults', 'total_infants',
       'total_children', 'route_type', 'group_pnr', 'expiration_time',
       'original_contact_group_id', 'original_booking_id', 'creation_hour',
       'total_passengers', 'id', 'pnr', 'expiration_time', 'seat_ssrs',
       'baggage_ssrs', 'meal_ssrs', 'insurance_ssrs', 'general_ssrs',
       'creation_time', 'modificatio

In [22]:
%%time
db = pymysql.connect(
        host = 'uia-checkin-prod-2020-11-20.cakfueupwjvh.eu-central-1.rds.amazonaws.com',
        user="root",
        password="68sP39Qn", 
        port=3306
      
    )
# Run  query in the database to write table names
tabs = []
try:
    # Print all the databases
    with db.cursor() as cur:
        cur.execute('SELECT table_name FROM information_schema.tables where table_name like "%sales_%"')
        for r in cur:
            #print(r)
            if not 'dcs' in r[0]:
                tabs.append(r[0])
finally:
    db.close()
        
print('YAYY!!!') 
print(len(tabs))
tabs[:3]

YAYY!!!
21
CPU times: user 198 µs, sys: 3.89 ms, total: 4.08 ms
Wall time: 19.7 ms


['xd2vf_anc_sales_Apr2019',
 'xd2vf_anc_sales_Aug2019',
 'xd2vf_anc_sales_Aug2020']

In [18]:
%%time
db = pymysql.connect(
        host = 'uia-checkin-prod-2020-11-20.cakfueupwjvh.eu-central-1.rds.amazonaws.com',
        user="root",
        password="68sP39Qn", 
        port=3306
      
    )
# Run  query in the database to union tables from the list
df = None

try:
    for sql_tab in tabs:
        if df is None:
            df = pd.read_sql('SELECT * FROM archive.'+ sql_tab, db)
        else:
            _df = pd.read_sql('SELECT * FROM archive.'+ sql_tab, db)
            df = df.append(_df)
            
finally:
    db.close()
print(df.shape)
 
print("YAYY!!")

(225222, 28)
YAYY!!


In [19]:
df.to_csv('Checkin.csv')

In [20]:
df.columns

Index(['id', 'date', 'creation_timestamp', 'request_from', 'pnr', 'first_name',
       'last_name', 'flight_number', 'flight_date', 'is_charter', 'tkne',
       'customer_id', 'segment_id', 'ssr_type', 'ssr_sub_type', 'quantity',
       'base_price', 'tax', 'total_price', 'currency', 'converted_price',
       'emd_number', 'form_of_payment', 'gateway_type', 'order_id',
       'psp_reference_number', 'receipt_url', 'status'],
      dtype='object')